In [21]:
# This script extracts the title, link, and short description of search results on Google 

#import all libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import time, sys, requests, random


# import libs, authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)


In [22]:
# Global variables

# paths/baseurls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
companyFilePath = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Companies/DigitalHealthWebscrape - Companies.csv'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Deliverables/'
googleurl = 'https://www.google.com/'
#ints 
adjustDenominator = 3
maxbackOff = 120
maxResult = 1
waitTime = 5
waitRun = 0.3

#bools
useURL = False
useName = False


In [23]:
def wait():
    pass
    global waitRun
    sleepTime = waitRun + random.uniform(0, 1)
    #print('sleepTime: ', sleepTime)
    time.sleep(sleepTime)
    waitRun = waitRun*2
    #print('waitRun: ', waitRun)
    if (waitRun >= maxbackOff):
        sys.exit("error: read from sheets quota exceeded")

In [24]:
# Helper function: reads in values from DigitalHealthWebscrape google sheet
# Need spreadsheet ID and the cell address where the value should be read in 
# returns the value 

def readinValue(cellLocation, sheetIndex):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndex)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

In [25]:
def readWebsites():
    links = []
    names = []
    df = pd.read_csv(companyFilePath)
    df['Name']=df['Name'].fillna(' ')
    df['Link']=df['Link'].fillna(' ')
    names = df['Name'].tolist()
    links = df['Link'].tolist()
    print(df)
    return links, names 
    

In [26]:
def readKeyWords():
    keywords = []
    row = 2;
    sheetIndex = 0
    isDone = False
    cellLocationColumn = 'C'
    while (isDone == False):
        cellLocationKeyWords = cellLocationColumn + str(row)
        try:
            keyword = readinValue(cellLocationKeyWords, sheetIndex)
            keywords.append(keyword)
            if(keyword == None):
                isDone = True
                break
            else:
                row += 1 
        except:
            wait()
    keywords = keywords[0:len(keywords)-1]
    print(keywords)
    return keywords

In [27]:
def readCalculateLiklihood():
    #print(links, names)
    sheetIndex = 0
    cellLocationLiklihood = 'B2'
    try:
        calculateLiklihoodInput = readinValue(cellLocationLiklihood, sheetIndex)
        if (calculateLiklihoodInput == 'Yes'):
            calculateLiklihood = True
        else:
            calculateLiklihood = False
    except:
        wait()
    return calculateLiklihood

In [28]:
def readInput():
    links, names = readWebsites()
    calculateLiklihood = readCalculateLiklihood()
    keywords = readKeyWords()
    return links, names, calculateLiklihood, keywords

In [29]:
# helper function to extract links from 1 page of results
def extractLinks(soup):
    links = []
    #Tags and classes
    linksTag = 'div'
    linksClass = 'yuRUbf'
    linksAttr = 'href'
    searchLinks = soup.find_all(linksTag, class_ = linksClass)
    for h in searchLinks:
        link = h.a.get(linksAttr)
        links.append(link)
    return links 

In [30]:
def extractTitles(soup):
    titles = [] 
    
    titlesClass = 'LC20lb MBeuO DKV0Md'
    titlesTag = 'h3'
    searchTitles = soup.find_all(titlesTag, class_= titlesClass)
    for h in searchTitles:
        titles.append(h.text)
    return titles

In [31]:
# helper function to extract texts from 1 page of results
def extractTexts(soup):
    texts = []
    textsClass = 'VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf'
    textsTag = 'div'
    searchText = soup.find_all(textsTag, class_= textsClass)
    for h in searchText:
        fullText = h.text
        try:
            splitText = fullText.split('— ', 1)
            text = splitText[1]
            texts.append(text);
        except:
            texts.append(fullText)
    return texts;

In [32]:
def calculateLiklihoods(links, keywords):
    #keywords = ['personalized', 'personalization', 'machine-learning', 'AI', 'Artificial Intelligence', '24/7', 'democratizing']
    liklihoods = []
    detectedWordsAll = []
    ignore = ['[document]', 'a', 'article', 'label', 'script', 'style']
    liklihoodDenom = len(keywords) - adjustDenominator
    for url in links: 
        detectedWords = []
        output = ''
        count = 0
        liklihood = 0

        res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        html_page = res.content
        soup = bs(html_page, 'html.parser')
        #print("Encoded method :" + url + ": ", soup.original_encoding)
        text = soup.find_all(text=True)
       
        for t in text:
            if t.parent.name not in ignore:
                output += '{} '.format(t)

        # analyze the webpage to detect keywords 
        outputSub = output.split(' ')
        #print(outputSub)
        for word in keywords: 
            if (word in outputSub or word.capitalize() in outputSub or word + '\n' in outputSub):
                #print('"' + word +'"' + ' detected')
                detectedWords.append(word)
                count +=1 
            #else:
                #print(word + ' is not there')
        liklihood = round((count / liklihoodDenom), 2)
        liklihoods.append(liklihood)
        detectedWordsAll.append(detectedWords)
        #detectedWordsAll = ', '.join(str(keyword) for keyword in detectedWordsAll)
    #print("detectedWordsAll", detectedWordsAll)
    return liklihoods, detectedWordsAll

In [33]:
# set the keyword you want to search for depending on whether link or name is given
# we find the search bar using its name attribute value (q)

def searchGoogle(index, linksInput, namesInput, googleurl, useURL, waitTime, driver):
    #print("index: ", index)
    #print("Number of Companies: " + str((len(linksInput))))
    #print("links: ", linksInput)
    #print("names: ", namesInput)
    siteURL = linksInput[index]
    #print("siteURL: ", siteURL)
    websiteName = namesInput[index]
    #print("websiteName: ", websiteName)
    driver.get(googleurl)
    searchBar = driver.find_element(By.NAME, 'q')
    
    # Booleans
    if (siteURL == ' '):
        useURL = False; 
        useName = True; 
    else:
        useURL = True; 
        useName = False; 

    # first we send our keyword to the search bar followed by the enter # key depending on using URL or website name 

    if (useURL):
        query = "site: " + siteURL
        print('query: ' + query)
        try:
            searchBar.send_keys(query)
            searchBar.send_keys('\n')
        except Exception as e : 
            WebDriverWait(driver, waitTime).until(EC.presence_of_element_located((By.NAME, 'q')))
            searchBar.send_keys(query)
            searchBar.send_keys('\n')
    else:
        query = websiteName
        print('query: ' + query)
        searchBar.send_keys(query)
        searchBar.send_keys('\n')

In [34]:
#capture links, header, and text
#pageInfo is a list of dictionaries for each page with keys/value pairs: header, link, text
# extract and load each page of results to pageInfo 
def parseHTML(driver, calculateLiklihood, keywords, maxResult, pageInfo):
    soup = bs(driver.page_source, 'html.parser')
    links = extractLinks(soup);
    texts = extractTexts(soup);
    titles = extractTitles(soup);
    if (calculateLiklihood):
        liklihoods, detectedWordsAll = calculateLiklihoods(links[0:1], keywords)
        #print('detectedWordsAllStringForm: ' + str(detectedWordsAll))
        #detectedWordsAll = ', '.join([str(keyword) for keyword in detectedWordsAll])
        #print("detectedWordsAll String Form: " + detectedWordsAll)
        pageInfo = addToPageInfo(titles, links, texts, maxResult, pageInfo, liklihoods, detectedWordsAll)
    else:
        pageInfo = addToPageInfo(titles, links, texts, maxResult, pageInfo, liklihoods = None, detectedWordsAll = None)

    return pageInfo

In [35]:
# helper function to add 1 page of results to pageInfo list
def addToPageInfo(titles, links, texts, maxResult, pageInfo, liklihoods, detectedWordsAll):
    index = 0;
    while (index < maxResult):
        #print("texts: ", texts)
        #print("liklihoods: ", liklihoods)
        #print("length of links list: " + str((len(linksInput)-1)))
        #print("links: ", links)
        #print("titles: ", titles)
        #print('detectedWordsAll', detectedWordsAll)
        #create new dictionary of each search results' title, link, and text
        if (liklihoods == None):
            pageInfo.append({"Title": titles[index], "Link": links[index], "About": texts[index]})
        else:
            #print("detectedWordsString: ", detectedWordsAll)
            detectedWords = str(detectedWordsAll[index])
            pageInfo.append({"Probability": liklihoods[index], "Title": titles[index], "Link": links[index], "About": texts[index],  "Detected Words": detectedWords})
        index += 1
    return pageInfo


In [36]:
def exportCSV(df, pathtoFile, fileName):
    # convert pageInfo to pandas dataframe and export as csv 
    df.to_csv(pathtoFile + fileName)

In [37]:
def updateSpreadSheet(df, sheetIndex):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndex)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    

In [38]:
def exportDeliverables(pageInfo, pathtoFile, fileName, calculateLiklihood):
    df = pd.DataFrame(pageInfo)
    if (calculateLiklihood):
        df = df.sort_values(by = ['Probability'], ascending=False)
    
    sheetIndex = 2
    #print(df.columns.values.tolist())
    #print(df.values.tolist())
    #print(pageInfo)
    exportCSV(df, pathtoFile, fileName)
    updateSpreadSheet(df, 2)
    return df 

In [39]:
def runExtraction():
    index = 0
    linksInput, namesInput, calculateLiklihood, keywords = readInput()
    print("Number of Companies: " + str((len(linksInput))))
    fileName = 'InputCompanies.csv'
    # Access chromedriver and determine path 
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    
    # list of dictionaries with key/value pairs: title, link, text
    # Contains all information for all search results 
    pageInfo = []

    while (index < len(linksInput)):
        searchGoogle(index, linksInput, namesInput, googleurl, useURL, waitTime, driver)
        for page in range(0, 1):
            pageInfo = parseHTML(driver, calculateLiklihood, keywords, maxResult, pageInfo)
    #print(pageInfo)
            index += 1
        time.sleep(waitRun)
    df = exportDeliverables(pageInfo, pathtoFile, fileName, calculateLiklihood)
    return df

In [40]:
# Run the script 
# implement exponential backoff algorithm to prevent exceeding read quota from sheets
startTime = time.time()
df = runExtraction()
endTime = time.time()
timeElapsed = endTime - startTime
print('Time Elapsed: ', timeElapsed)
df.head()


   Unnamed: 0             Name Link
0   Companies     OneRemission     
1         NaN           Youper     
2         NaN   Babylon Health     
3         NaN  Florence Health     
4         NaN        Healthily     
5         NaN       Ada Health     
6         NaN          Sensely     
7         NaN      Buoy Health     
8         NaN      Infermedica     
9         NaN            GYANT     
10        NaN           Woebot     
['personalized', 'personalization', 'machine-learning', 'AI', 'Artificial Intelligence', 'A.I.', 'democratizing', '24/7', 'digital assistant ', 'on-demand', 'better access']
Number of Companies: 11


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=98.0.4758.109)
Stacktrace:
0   chromedriver                        0x0000000104b47ee9 chromedriver + 5013225
1   chromedriver                        0x0000000104ad31d3 chromedriver + 4534739
2   chromedriver                        0x00000001046a9a68 chromedriver + 170600
3   chromedriver                        0x0000000104699991 chromedriver + 104849
4   chromedriver                        0x000000010469ae42 chromedriver + 110146
5   chromedriver                        0x0000000104693c12 chromedriver + 80914
6   chromedriver                        0x00000001046aaed3 chromedriver + 175827
7   chromedriver                        0x000000010470d75c chromedriver + 579420
8   chromedriver                        0x00000001046fb6d3 chromedriver + 505555
9   chromedriver                        0x00000001046d176e chromedriver + 333678
10  chromedriver                        0x00000001046d2745 chrom